In [1]:
import sys
import subprocess
import importlib
import threading
import socket
import time
import struct
import os
import csv

# ---------------------------------------------------------
# 1. 패키지 자동 설치 함수
# ---------------------------------------------------------
def install_package(module_name, package_name=None):
    if package_name is None:
        package_name = module_name
    try:
        importlib.import_module(module_name)
    except ImportError:
        print(f"Installing {package_name} ...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
            print(f"{package_name} installation completed")
        except subprocess.CalledProcessError as e:
            print(f"{package_name} installation failed (exit code {e.returncode})")
            sys.exit(1)

print("Checking required packages...")
install_package("numpy")
install_package("cv2", "opencv-python")
install_package("PyQt5")

import numpy as np
import cv2
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
from PyQt5 import QtWidgets, QtGui

# 조이스틱 모듈 임포트 (같은 폴더에 myjoystick.py가 있어야 함)
try:
    from myjoystick import MyJoystick
except ImportError:
    print("Error: 'myjoystick.py' file not found in the same directory.")
    sys.exit(1)

# ---------------------------------------------------------
# 2. 윈도우 소켓 버그 수정용 함수 (recvall)
# ---------------------------------------------------------
def recvall(sock, count):
    buf = b''
    while len(buf) < count:
        try:
            newbuf = sock.recv(count - len(buf))
            if not newbuf: return None
            buf += newbuf
        except socket.timeout:
            raise
        except BlockingIOError:
            continue
    return buf

# ---------------------------------------------------------
# 전역 변수 설정
# ---------------------------------------------------------
HOST_CAM = '192.168.137.220'
PORT_CAM = 80
PORT_MOT = 81

client_cam = None
client_mot = None

running = False
label_widget = None
camera_thread = None

dirname = ""
f_csv = None
wr = None
labels_list = ["_0_forward", "_1_right", "_2_left", "_3_stop"]

# 조이스틱 데이터 저장을 위한 전역 변수
g_rl = 0 

# ---------------------------------------------------------
# [기능] 소켓 연결 함수
# ---------------------------------------------------------
def try_connect_esp32(ip_address):
    global client_cam, client_mot
    print(f"Connecting to ESP32 ({ip_address})...")
    
    if client_cam:
        try: client_cam.close()
        except: pass
    if client_mot:
        try: client_mot.close()
        except: pass

    try:
        cam_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        mot_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
        cam_sock.settimeout(3)
        mot_sock.settimeout(3)

        cam_sock.connect((ip_address, PORT_CAM))
        mot_sock.connect((ip_address, PORT_MOT))
        
        cam_sock.settimeout(5)
        mot_sock.settimeout(5)
        
        client_cam = cam_sock
        client_mot = mot_sock
        print("Connected successfully!")
        return True
        
    except Exception as e:
        print(f"Connection Failed: {e}")
        return False

# ---------------------------------------------------------
# [기능] 폴더 생성 함수
# ---------------------------------------------------------
def createNewFolder():
    global dirname, f_csv, wr
    if f_csv:
        try: f_csv.close()
        except: pass

    dirname = "data_%f" %(time.time())
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    
    for label in labels_list:
        path = os.path.join(dirname, label)
        if not os.path.exists(path):
            os.mkdir(path)

    f_csv = open(os.path.join(dirname, "0_road_labels.csv"),'w', newline='')
    wr = csv.writer(f_csv)
    wr.writerow(["file","label"])
    print(f"--> New data folder created: {dirname}")

# ---------------------------------------------------------
# 카메라 쓰레드
# ---------------------------------------------------------
def camMain():
    global running, g_rl, dirname, f_csv, wr
    
    t_prev = time.time()
    cnt_frame = 0
    cnt_frame_total = 0
    
    DISPLAY_WIDTH = 320
    DISPLAY_HEIGHT = 240
    
    # UI 준비 대기
    while label_widget is None and running:
        time.sleep(0.1)
    
    try:
        if label_widget: label_widget.resize(DISPLAY_WIDTH, DISPLAY_HEIGHT)
    except: pass

    while running:
        if client_cam is None:
            time.sleep(1)
            continue

        try:
            # 영상 요청
            cmd = 12
            cmd = struct.pack('B', cmd)
            client_cam.sendall(cmd) 

            # 헤더 수신
            data_len_bytes = recvall(client_cam, 4)
            if not data_len_bytes: continue
            data_len = struct.unpack('I', data_len_bytes)[0]
            
            # 데이터 수신
            data = recvall(client_cam, data_len)
            if not data: continue

            # 이미지 디코딩
            np_data = np.frombuffer(data, dtype='uint8')
            frame = cv2.imdecode(np_data, 1)
            if frame is None: continue

            # 전처리
            frame = cv2.rotate(frame, cv2.ROTATE_180) 
            frame_resized = cv2.resize(frame, (DISPLAY_WIDTH, DISPLAY_HEIGHT), interpolation=cv2.INTER_LINEAR)
            
            h, w, c = frame_resized.shape
            qImg = QtGui.QImage(frame_resized.data, w, h, w*c, QtGui.QImage.Format_RGB888)
            pixmap = QtGui.QPixmap.fromImage(qImg.rgbSwapped())
            
            if label_widget is not None:
                label_widget.setPixmap(pixmap)
            
            # --- 데이터 저장 로직 (원본 로직 유지 + 폴더 동적 처리) ---
            rl_val = g_rl
            collect_data = (rl_val & 4) >> 2
            label_idx = rl_val & 3

            if collect_data == 1 and f_csv is not None and not f_csv.closed:
                road_file = "%f.png" %(time.time())
                save_path = os.path.join(dirname, labels_list[label_idx])
                
                # 이미지 저장
                cv2.imwrite(os.path.join(save_path, road_file), frame_resized)
                
                # CSV 저장
                if wr:
                    wr.writerow([os.path.join(labels_list[label_idx], road_file), label_idx])
                    f_csv.flush()           
                cnt_frame_total += 1

            cnt_frame += 1
            t_now = time.time()
            if t_now - t_prev >= 1.0 :
                print(f"FPS: {cnt_frame}, Total Saved: {cnt_frame_total}, Current Label: {label_idx} ({labels_list[label_idx]})")
                t_prev = t_now
                cnt_frame = 0
                
        except socket.timeout:
            print("Socket timed out. Waiting...")
            time.sleep(0.5)
            continue
        except Exception as e:
            print(f"Cam Error: {e}")
            break

# ---------------------------------------------------------
# 조이스틱 콜백 함수 (원본 로직 유지)
# ---------------------------------------------------------
def cbJoyPos(joystickPosition):
    global g_rl
    
    if client_mot is None: return

    posX, posY = joystickPosition
        
    # --- [원본 보존] 자동차 방향 로직 시작 ---
    right, left = -1, -1
    collect_data = 1
    if posY < -0.5:
        right, left = 1, 1 # brake
    elif posY > 0.15 :
        if -0.15 <= posX <= 0.15 :
            right, left = 0, 0 # forward
        elif posX < -0.15 : 
            right, left = 1, 0 # left
        elif posX > 0.15 :
            right, left = 0, 1 # right
    else : # -0.5 <= posY <= 0.15
        right, left = 1, 1 # stop driving
        collect_data = 0
    
    rl = collect_data << 2 | right << 1 | left
    # --- [원본 보존] 자동차 방향 로직 끝 ---

    g_rl = rl
    
    try:
        rl_byte = struct.pack('B', rl)
        client_mot.sendall(rl_byte)
    except:
        pass

# ---------------------------------------------------------
# 메인 윈도우 클래스
# ---------------------------------------------------------
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('RC Car Joystick Controller')
        self.setGeometry(100, 100, 350, 500)

        cw = QWidget()
        self.setCentralWidget(cw)
        layout = QGridLayout() # 기존 레이아웃 유지
        cw.setLayout(layout)

        # 1. 화면 Label
        global label_widget
        label_widget = QLabel("Waiting for Video...")
        label_widget.setAlignment(Qt.AlignCenter)
        label_widget.setStyleSheet("background-color: black; color: white;")
        label_widget.setScaledContents(True)
        # 화면을 상단에 배치 (row 0)
        layout.addWidget(label_widget, 0, 0, 1, 2) # colspan 2

        # 2. 조이스틱
        self.joystick = MyJoystick(cbJoyPos)
        # 조이스틱을 중간에 배치 (row 1)
        layout.addWidget(self.joystick, 1, 0, 1, 2)

        # 3. 편의 기능 버튼들 (row 2, 3)
        self.btn_new_folder = QPushButton("Make New Folder (N)")
        self.btn_new_folder.setFixedHeight(40)
        self.btn_new_folder.clicked.connect(self.on_new_folder_clicked)
        layout.addWidget(self.btn_new_folder, 2, 0, 1, 2)
        
        self.btn_change_ip = QPushButton(f"Change IP (Current: {HOST_CAM})")
        self.btn_change_ip.setFixedHeight(40)
        self.btn_change_ip.clicked.connect(self.on_change_ip_clicked)
        layout.addWidget(self.btn_change_ip, 3, 0, 1, 2)

        # 안내 문구
        info_label = QLabel("Use Joystick to Drive.\n'N': New Folder, 'ESC': Exit")
        info_label.setAlignment(Qt.AlignCenter)
        layout.addWidget(info_label, 4, 0, 1, 2)

    def on_new_folder_clicked(self):
        createNewFolder()
        QMessageBox.information(self, "New Folder", f"New folder created:\n{dirname}")

    def on_change_ip_clicked(self):
        global HOST_CAM, running, camera_thread
        
        text, ok = QInputDialog.getText(self, 'Change IP', 'Enter ESP32 IP Address:', text=HOST_CAM)
        
        if ok and text:
            print("Reconnecting to new IP...")
            HOST_CAM = text
            self.btn_change_ip.setText(f"Change IP (Current: {HOST_CAM})")
            
            running = False
            if camera_thread:
                camera_thread.join(timeout=1.0)
            
            if try_connect_esp32(HOST_CAM):
                QMessageBox.information(self, "Success", "Connected to new IP!")
                running = True
                camera_thread = threading.Thread(target=camMain)
                camera_thread.setDaemon(True)
                camera_thread.start()
            else:
                QMessageBox.warning(self, "Failed", "Could not connect to the new IP.")

    def keyPressEvent(self, event):
        key = event.key()
        if key == Qt.Key_Escape:
            self.close()
            return
        if key == Qt.Key_N:
            self.on_new_folder_clicked()
            return

    def closeEvent(self, event):
        print("Window closing...")
        event.accept()

# ---------------------------------------------------------
# 실행부 (안전한 시작 로직)
# ---------------------------------------------------------
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
app.setStyle(QStyleFactory.create("Cleanlooks"))

# 최초 접속 시도 루프
connected = False
while not connected:
    connected = try_connect_esp32(HOST_CAM)
    
    if not connected:
        text, ok = QInputDialog.getText(None, 'Connection Failed', 
                                        'Failed to connect.\nEnter ESP32 IP Address:', 
                                        text=HOST_CAM)
        if ok and text:
            HOST_CAM = text 
        else:
            print("User cancelled connection.")
            sys.exit()

if connected:
    createNewFolder() # 시작 시 폴더 생성
    
    mw = MainWindow()
    mw.show()

    running = True
    camera_thread = threading.Thread(target=camMain)
    camera_thread.setDaemon(True) 
    camera_thread.start()

    print("Joystick Control Running. Press ESC to quit.")
    
    try:
        app.exec_()
    except SystemExit:
        pass
    finally:
        print("Cleaning up resources...")
        running = False
        try: client_cam.close() 
        except: pass
        try: client_mot.close() 
        except: pass
        if f_csv:
            try: f_csv.close()
            except: pass
        try: del mw
        except: pass
        try: del app
        except: pass
        print("Done.")

Checking required packages...
Connecting to ESP32 (192.168.137.220)...
Connection Failed: timed out
Connecting to ESP32 (192.168.137.230)...
Connected successfully!
--> New data folder created: data_1764943302.533215
Joystick Control Running. Press ESC to quit.


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_27344\3212318897.py:386: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  camera_thread.setDaemon(True)


FPS: 28, Total Saved: 0, Current Label: 3 (_3_stop)
FPS: 28, Total Saved: 15, Current Label: 2 (_2_left)
FPS: 28, Total Saved: 39, Current Label: 3 (_3_stop)
FPS: 28, Total Saved: 53, Current Label: 3 (_3_stop)
FPS: 26, Total Saved: 78, Current Label: 0 (_0_forward)
FPS: 28, Total Saved: 106, Current Label: 0 (_0_forward)
FPS: 28, Total Saved: 120, Current Label: 3 (_3_stop)
FPS: 28, Total Saved: 120, Current Label: 3 (_3_stop)
FPS: 28, Total Saved: 132, Current Label: 3 (_3_stop)
FPS: 28, Total Saved: 132, Current Label: 3 (_3_stop)
FPS: 28, Total Saved: 132, Current Label: 3 (_3_stop)
FPS: 28, Total Saved: 141, Current Label: 3 (_3_stop)
FPS: 28, Total Saved: 143, Current Label: 1 (_1_right)
FPS: 28, Total Saved: 171, Current Label: 1 (_1_right)
FPS: 25, Total Saved: 196, Current Label: 2 (_2_left)
FPS: 28, Total Saved: 224, Current Label: 2 (_2_left)
FPS: 28, Total Saved: 252, Current Label: 2 (_2_left)
FPS: 26, Total Saved: 278, Current Label: 2 (_2_left)
FPS: 28, Total Saved: 304,